In [34]:
import polars as pl
import numpy as np
from pathlib import Path
from polimi.utils._custom import load_history, load_articles, load_behaviors

In [35]:
base_path = Path('dataset')
type = 'demo'
split = 'train'
CSV_STORAGE = Path('dataset/graph').joinpath(type)

a = load_articles(base_path, type, split)
h = load_history(base_path, type, split)
b = load_behaviors(base_path, type, split)

# Fix

In [36]:
a = a.with_columns(pl.when(pl.col('category_str') == 'bibliotek').then(2504).otherwise(pl.col('category')).alias('category'))
a = a.with_columns(pl.when(pl.col('category') == 68).then(pl.lit("video")).otherwise(pl.col('category_str')).alias('category_str'))

#creation homepage node
home_page_article_dict = {k: None for k in a.columns}
home_page_article_id = 0
home_page_article_dict['article_id'] = home_page_article_id
home_page_article_dict['title'] = 'homepage'
home_page_article_dict['article_type'] = 'homepage'
a = a.vstack(pl.DataFrame(home_page_article_dict, schema=a.schema))

#link null article id in impression to homepage node
b = b.with_columns(pl.col('article_id').fill_null(home_page_article_id))


## Creation Neo4j CSV

In [37]:
b_tmp = b.select('user_id', 'gender', 'postcode', 'age').group_by('user_id').first()
u_data = h.select('user_id').join(b_tmp, on='user_id', how='left').sort('user_id').unique('user_id')
u_data = u_data.with_columns(pl.lit('User').alias(':LABEL'))
u_data=u_data.rename({'user_id': 'id:ID(User)'})

u_data.write_csv(CSV_STORAGE / 'user_data.csv')
u_data.head(3)

id:ID(User),gender,postcode,age,:LABEL
u32,i8,i8,i8,str
11313,null,null,null,"""User"""
13538,null,null,null,"""User"""
15430,null,null,null,"""User"""


In [38]:
i_data = b.drop('article_id', 'article_ids_inview','article_ids_clicked', 'user_id',
                'gender', 'postcode', 'age')\
          .sort('impression_id')


i_data = i_data.with_columns(pl.lit('Impression').alias(':LABEL'))
i_data=i_data.rename({'impression_id': 'id:ID(Impression)'})
i_data.write_csv(CSV_STORAGE / 'impression_data.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
i_data.head(3)

id:ID(Impression),impression_time,read_time,scroll_percentage,device_type,is_sso_user,is_subscriber,session_id,next_read_time,next_scroll_percentage,:LABEL
u32,datetime[μs],f32,f32,i8,bool,bool,u32,f32,f32,str
48401,2023-05-21 21:06:50,21.0,null,2,false,false,21,16.0,27.0,"""Impression"""
152513,2023-05-24 07:31:26,30.0,100.0,1,false,false,298,2.0,48.0,"""Impression"""
155390,2023-05-24 07:30:33,45.0,null,1,false,false,401,215.0,100.0,"""Impression"""


In [39]:
topic = a.select(pl.col('topics').alias('topic')).explode('topic').unique().sort('topic').drop_nulls()
topic = topic.insert_column(0, pl.Series("topic_id", np.arange(topic.shape[0])))
topic = topic.with_columns(pl.lit('Topic').alias(':LABEL'))
topic=topic.rename({'topic': 'name', 'topic_id': 'id:ID(Topic)'})
topic.write_csv(CSV_STORAGE / 'topic_data.csv')
topic.head(3)

id:ID(Topic),name,:LABEL
i64,str,str
0,"""Ansættelsesfor…","""Topic"""
1,"""Bandekriminali…","""Topic"""
2,"""Bedrageri""","""Topic"""


In [40]:
entity_group = a.select(pl.col('entity_groups').alias('entity_group')).explode('entity_group').unique().sort('entity_group').drop_nulls()
entity_group = entity_group.insert_column(0, pl.Series("entity_group_id", np.arange(entity_group.shape[0])))
entity_group = entity_group.with_columns(pl.lit('Entity').alias(':LABEL'))
entity_group = entity_group.rename({'entity_group': 'name', 'entity_group_id': 'id:ID(Entity)'})
entity_group.write_csv(CSV_STORAGE / 'entity_data.csv')
entity_group.head(3)

id:ID(Entity),name,:LABEL
i64,str,str
0,"""EVENT""","""Entity"""
1,"""LOC""","""Entity"""
2,"""MISC""","""Entity"""


In [41]:
cats = a.select(pl.col('category').alias('category_id'), pl.col('category_str').alias('name')).unique().drop_nulls()
ids_cats = cats.select('category_id').to_numpy().reshape(-1)
sub_cats = a.select(pl.col('subcategory').alias('category_id')).explode('category_id').unique().drop_nulls()
sub_cats = sub_cats.filter(~pl.col('category_id').is_in(cats.select('category_id'))).with_columns(pl.lit(None).alias('name'))
category = cats.vstack(sub_cats).sort('category_id')
category = category.with_columns(pl.lit('Category').alias(':LABEL')).sort('category_id').unique()
category=category.rename({'category_id': 'id:ID(Category)'})
category.write_csv(CSV_STORAGE / 'category_data.csv')
category.head(3)

id:ID(Category),name,:LABEL
i16,str,str
136,null,"""Category"""
140,"""krimi""","""Category"""
196,null,"""Category"""


In [42]:
a_to_category = a.select('article_id', pl.lit(False).alias('is_sub'), pl.col('category').alias('category_id')).unique().drop_nulls()
a_to_sub_category = a.select('article_id', pl.lit(True).alias('is_sub'), pl.col('subcategory').alias('category_id')).explode('category_id').unique().drop_nulls()

a_to_category = a_to_category.vstack(a_to_sub_category).rename({'article_id': ':START_ID(Article)', 'category_id': ':END_ID(Category)'})\
    .with_columns(pl.lit('IS_IN').alias(':TYPE'))
a_to_category.write_csv(CSV_STORAGE / 'article_to_category.csv')
a_to_category.head(3)

:START_ID(Article),is_sub,:END_ID(Category),:TYPE
i32,bool,i16,str
3037230,false,142,"""IS_IN"""
3993395,false,140,"""IS_IN"""
4008213,false,414,"""IS_IN"""


In [43]:
a_to_topic = a.select('article_id', pl.col('topics').alias('name')).explode('name').sort('article_id').unique().drop_nulls()
a_to_topic = a_to_topic.join(topic, on='name', how='left').select('article_id', pl.col('id:ID(Topic)').alias('topic_id'))\
    .with_columns(pl.lit('IS_IN').alias(':TYPE'))
a_to_topic = a_to_topic.rename({'article_id': ':START_ID(Article)', 'topic_id': ':END_ID(Topic)'})
a_to_topic.write_csv(CSV_STORAGE / 'article_to_topic.csv')
a_to_topic.head(3)

:START_ID(Article),:END_ID(Topic),:TYPE
i32,i64,str
3073151,10,"""IS_IN"""
3073151,20,"""IS_IN"""
3073151,52,"""IS_IN"""


In [44]:
a_to_entity = a.select('article_id', pl.col('entity_groups').alias('name')).explode('name').sort('article_id').unique().drop_nulls()
a_to_entity = a_to_entity.join(entity_group, on='name', how='left').select('article_id', pl.col('id:ID(Entity)').alias('entity_id'))\
        .with_columns(pl.lit('IS_IN').alias(':TYPE'))

a_to_entity = a_to_entity.rename({'article_id': ':START_ID(Article)', 'entity_id': ':END_ID(Entity)'})
a_to_entity.write_csv(CSV_STORAGE / 'article_to_entity.csv')
a_to_entity.head(3)

:START_ID(Article),:END_ID(Entity),:TYPE
i32,i64,str
5408452,4,"""IS_IN"""
5996581,4,"""IS_IN"""
6018488,3,"""IS_IN"""


In [45]:
a_data = a.drop('image_ids', 'ner_clusters', 'entity_groups', 'topics', 'subcategory', 
                'category', 'category_str')\
                    .with_columns(pl.col('title').str.replace_all('\n', ''))\
                    .with_columns(pl.col('subtitle').str.replace_all('\n', ''))\
                    .with_columns(pl.col('body').str.replace_all('\n', ''))\
                    .with_columns(pl.lit('Article').alias(':LABEL'))\
                    .sort('article_id')

a_data = a_data.rename({'article_id': 'id:ID(Article)'})
a_data.write_csv(CSV_STORAGE / 'article_data.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
a_data.head(3)

id:ID(Article),title,subtitle,last_modified_time,premium,body,published_time,article_type,url,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,:LABEL
i32,str,str,datetime[μs],bool,str,datetime[μs],str,str,i32,i32,f32,f32,str,str
0,"""homepage""",null,null,null,null,null,"""homepage""",null,null,null,null,null,null,"""Article"""
3037230,"""Ishockey-spill…","""ISHOCKEY: Isho…",2023-06-29 06:20:57,false,"""Ambitionerne o…",2003-08-28 08:55:00,"""article_defaul…","""https://ekstra…",null,null,null,0.9752,"""Negative""","""Article"""
3044020,"""Prins Harry tv…","""Hoffet tvang P…",2023-06-29 06:21:16,false,"""Den britiske t…",2005-06-29 08:47:00,"""article_defaul…","""https://ekstra…",null,null,null,0.7084,"""Negative""","""Article"""


In [46]:
u_to_i = b.select('user_id', 'impression_id') \
    .sort('user_id').unique().drop_nulls() \
    .with_columns(pl.lit('GENERATE').alias(':TYPE'))
    
u_to_i = u_to_i.rename({'user_id': ':START_ID(User)', 'impression_id': ':END_ID(Impression)'})
u_to_i.write_csv(CSV_STORAGE / 'user_to_impression.csv')
u_to_i.head(3)

:START_ID(User),:END_ID(Impression),:TYPE
u32,u32,str
11313,487155599,"""GENERATE"""
13538,476960980,"""GENERATE"""
13538,574878371,"""GENERATE"""


In [47]:
i_to_a = b.select('impression_id', pl.col('article_id')).sort('impression_id').unique().drop_nulls()\
    .with_columns(pl.lit('ON').alias(':TYPE'))
i_to_a = i_to_a.rename({'impression_id': ':START_ID(Impression)', 'article_id': ':END_ID(Article)'})
i_to_a.write_csv(CSV_STORAGE / 'impression_to_article.csv')
i_to_a.head(3)

:START_ID(Impression),:END_ID(Article),:TYPE
u32,i32,str
369958,0,"""ON"""
369959,0,"""ON"""
374777,9779289,"""ON"""


In [48]:
i_to_a_clicked = b.select('impression_id', pl.col('article_ids_clicked').alias('article_id')).explode('article_id').sort('impression_id')\
    .with_columns(pl.lit('CLICK').alias(':TYPE'))
i_to_a_clicked = i_to_a_clicked.rename({'impression_id': ':START_ID(Impression)', 'article_id': ':END_ID(Article)'})
i_to_a_clicked.write_csv(CSV_STORAGE / 'impression_to_article_clicked.csv')
i_to_a_clicked.head(3)

:START_ID(Impression),:END_ID(Article),:TYPE
u32,i32,str
48401,9759966,"""CLICK"""
152513,9778661,"""CLICK"""
155390,9777856,"""CLICK"""


In [49]:
i_to_a_inview = b.select('impression_id', pl.col('article_ids_inview').alias('article_id')).explode('article_id').sort('impression_id').drop_nulls()
i_to_a_inview = i_to_a_inview.rename({'impression_id': ':START_ID(Impression)', 'article_id': ':END_ID(Article)'})\
    .with_columns(pl.lit('INVIEW').alias(':TYPE'))
i_to_a_inview.write_csv(CSV_STORAGE / 'impression_to_article_inview.csv')
i_to_a_inview.head(3)

:START_ID(Impression),:END_ID(Article),:TYPE
u32,i32,str
48401,9774516,"""INVIEW"""
48401,9771051,"""INVIEW"""
48401,9770028,"""INVIEW"""


### History

In [ ]:
batch = 0
batch_size=100
user_ids = h.select('user_id')

h_final = None
for batch in range(0, len(user_ids), batch_size):
    
    user_ids_batch = user_ids[batch:batch+batch_size]
    h_data = h.filter(pl.col('user_id').is_in(user_ids[batch:batch+batch_size]))\
    .explode('impression_time_fixed', 'article_id_fixed', 'scroll_percentage_fixed', 'read_time_fixed')

    h_data = h_data.with_columns(pl.Series(np.arange(-1, len(h_data) - 1)).cast(pl.Int32).alias('id_l'))
    h_data = h_data.with_columns(pl.Series(np.arange(len(h_data))).cast(pl.Int32).alias('id_r'))
    h_data = h_data.join(h_data, left_on=['user_id', 'id_r'], right_on=['user_id', 'id_l'], how='left')

    h_data = h_data.select(
        pl.col('article_id_fixed').alias(':START_ID(Article)'),
        'user_id', 
        pl.col('impression_time_fixed').alias('impression_time'),
        pl.col('scroll_percentage_fixed').alias('scroll_percentage'),
        pl.col('read_time_fixed').alias('read_time'),
        pl.col('article_id_fixed_right').alias(':END_ID(Article)'),
        pl.lit('FOLLOW').alias(':TYPE')
    ).drop_nulls(':END_ID(Article)')
    
    if h_final is None:
        h_final = h_data
    else:
        h_final = h_final.vstack(h_data)
    

h_final.write_csv(CSV_STORAGE / 'history_article_to_article.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
h_final.head(3)

:START_ID(Article),user_id,impression_time,scroll_percentage,read_time,:END_ID(Article),:TYPE
i32,u32,datetime[μs],f32,f32,i32,str
9735579,10029,2023-04-28 06:16:57,23.0,28.0,9739888,"""FOLLOW"""
9739888,10029,2023-04-28 06:17:31,69.0,24.0,9739471,"""FOLLOW"""
9739471,10029,2023-04-28 06:18:12,27.0,11.0,9739864,"""FOLLOW"""


In [ ]:
h_data = h.select(pl.col('user_id').alias(':START_ID(User)'), 
                  pl.col('impression_time_fixed').list.first().alias('impression_time'),
                  pl.col('scroll_percentage_fixed').list.first().alias('scroll_percentage'),
                  pl.col('read_time_fixed').list.first().alias('read_time'),
                  pl.col('article_id_fixed').list.first().alias(':END_ID(Article)'),
                  pl.lit('FOLLOW').alias(':TYPE'))

h_data.write_csv(CSV_STORAGE / 'history_user_to_article.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
h_data.head(3)

:START_ID(User),impression_time,scroll_percentage,read_time,:END_ID(Article),:TYPE
u32,datetime[μs],f32,f32,i32,str
10029,2023-04-28 06:16:57,23.0,28.0,9735579,"""FOLLOW"""
10033,2023-04-27 11:11:32,33.0,2.0,9738139,"""FOLLOW"""
10034,2023-04-30 09:46:57,null,21.0,9742693,"""FOLLOW"""


In [ ]:
' '.join(
    [
        'bin/neo4j-admin database import full',
        #nodes
        f'--nodes=User=import/{type}/user_data.csv',
        f'--nodes=Article=import/{type}/article_data.csv',
        #relationships
        f'--relationships=import/{type}/history_user_to_article.csv',
        f'--relationships=import/{type}/history_article_to_article.csv',
        
        '--overwrite-destination',
        f'newsdb-history-{type}'
        ])

'bin/neo4j-admin database import full --nodes=User=import/large/user_data.csv --nodes=Article=import/large/article_data.csv --relationships=import/large/history_user_to_article.csv --relationships=import/large/history_article_to_article.csv --overwrite-destination newsdb-history-large'

### Load neo4j-admin-script

In [58]:
from polimi.utils._graph import create_neo4j_admin_script

create_neo4j_admin_script('large')

'bin/neo4j-admin database import full --nodes=import/large/user_data.csv --nodes=import/large/impression_data.csv --nodes=import/large/article_data.csv --nodes=import/large/category_data.csv --nodes=import/large/entity_data.csv --nodes=import/large/topic_data.csv --relationships=import/large/user_to_impression.csv --relationships=import/large/impression_to_article.csv --relationships=import/large/impression_to_article_inview.csv --relationships=import/large/impression_to_article_clicked.csv --relationships=import/large/article_to_topic.csv --relationships=import/large/article_to_entity.csv --relationships=import/large/article_to_category.csv --overwrite-destination newsdb-large'

## Contraints

In [60]:
from neo4j import GraphDatabase
from polimi.utils._graph import add_constraint
from dotenv import load_dotenv
import os

load_dotenv()

# Retrieve Neo4j connection details
URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_AUTH_USER")
PASSW = os.getenv("NEO4J_AUTH_PASSWORD")

# Establish connection
driver = GraphDatabase.driver(URI, auth=(USER, PASSW), database=f'newsdb-')

print(driver.verify_connectivity())
add_constraint(driver, 'Article', 'id')
add_constraint(driver, 'Impression', 'id')
add_constraint(driver, 'User', 'id')
add_constraint(driver, 'Topic', 'id')
add_constraint(driver, 'Category', 'id')
add_constraint(driver, 'Entity', 'id')


driver.close()

None
{}
{}
{}
{}
{}
{}
